In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [2]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

爬取url：https://www.sciencedirect.com/journal/journal-of-public-economics/vol/200/suppl/C

爬取内容：一级页面上想抓每条记录的题目，作者，每个连接页面的二级页面想把关键词和英文摘要抓出来



In [ ]:
#如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install lxml -t /home/aistudio/external-libraries

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 17.1MB/s eta 0:00:01
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 5.5MB 14.4MB/s eta 0:00:01


In [1]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [2]:
import json
import re
import requests
import datetime
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import random
import time #时间处理模块
import argparse
import copy
import ast
import os

In [3]:
def getinfo(url):
    '''
    请求url接口，返回soup信息
    '''

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    requests.adapters.DEFAULT_RETRIES = 1
    s = requests.session()    
    s.keep_alive = False
    re = s.get(url, headers=headers)
    soup = BeautifulSoup(re.text, 'html.parser')
    return soup

In [ ]:


def saveInfoToFile1(volume1,volume2):
    '''
    解析数据，返回需要的数据articles
    volume1:起始期目号
    volume2:结束期目号+1
    '''
    articles = []
    Keywords_title = "Keywords"
    jelClassification_title = "JEL"
    while(volume1!=volume2):
        #url = 'https://www.sciencedirect.com/journal/journal-of-public-economics/vol/{}/suppl/C'
        url = 'https://www.sciencedirect.com/journal/economia/vol/{}/issue/1'
        url = url.format(str(volume1))
        soup = getinfo(url)    
        lis = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')
        for li in lis:
            article={}
            article['title'] = li.find('span',class_='js-article-title').text
            article['link'] = 'https://www.sciencedirect.com'+li.find('a', class_='anchor article-content-title u-margin-xs-top u-margin-s-bottom')['href']
            try:
                article['authors'] = li.find('div',class_='text-s u-clr-grey8 js-article__item__authors').text
            except AttributeError:
                article['authors'] = ''
            soup1 = getinfo(article['link'])
            try:
                article['abstract']=soup1.findAll('div', class_="abstract author")[0].find_all('div')[0].text
            except:
                article['abstract']=''
            article['期刊页码']= soup1.findAll('div',class_='text-xs')[0].text
            article['jelClassification'] = '无'
            keywords_divs=soup1.findAll('div', class_="keywords-section")
            for keywords_div in  keywords_divs:           
                #对当前节点前面的标签和字符串进行查找
                div_title = keywords_div.find_all('h2')[0].text
                if(Keywords_title in div_title):
                    keywords_list = [keyword.text for keyword in keywords_div]
                    article['keywords'] = ','.join(keywords_list[1:])
                if(jelClassification_title in div_title):
                    jelClassification_list = [keyword.text for keyword in keywords_div]
                    article['jelClassification'] = ','.join(jelClassification_list[1:])
            articles.append(article)
            #time.sleep(np.random.randint(0,5)) #在2秒~10秒钟随意切换数值，降低爬取频率，防止被封杀
        volume1 -= 1
        
    return articles

In [4]:
def saveInfoToFile2(volume1,volume2,url_sub,page,url_origi):
    '''
    解析数据，返回需要的数据
    volume1:起始期目号, 最大的
    volume2:结束期目号+1
    url_sub:网址中间类别
    page：issue后面的最大数字
    '''
    #url_origi = 'https://www.sciencedirect.com/journal/{}/vol/{}/issue/{}'
    articles = []
    Keywords_title = "Keywords"
    jelClassification_title = "JEL"
    for volume_tmp in range(volume1,volume2,-1):
        for page_tmp in range(page,0,-1):    
            url = url_origi.format(url_sub,str(volume_tmp),str(page_tmp))
            print(url)
            soup = getinfo(url)    
            lis = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')
            for li in lis:
                article={}
                article['title'] = li.find('span',class_='js-article-title').text
                article['link'] = 'https://www.sciencedirect.com'+li.find('a', class_='anchor article-content-title u-margin-xs-top u-margin-s-bottom')['href']
                try:
                    article['authors'] = li.find('div',class_='text-s u-clr-grey8 js-article__item__authors').text
                except AttributeError:
                    article['authors'] = ''
                soup1 = getinfo(article['link'])
                try:
                    article['abstract']=soup1.findAll('div', class_="abstract author")[0].find_all('div')[0].text
                except:
                    article['abstract']=''
                article['期刊页码']= soup1.findAll('div',class_='text-xs')[0].text
                article['jelClassification'] = '无'
                keywords_divs=soup1.findAll('div', class_="keywords-section")
                for keywords_div in  keywords_divs:           
                    #对当前节点前面的标签和字符串进行查找
                    div_title = keywords_div.find_all('h2')[0].text
                    if(Keywords_title in div_title):
                        keywords_list = [keyword.text for keyword in keywords_div]
                        article['keywords'] = ','.join(keywords_list[1:])
                    if(jelClassification_title in div_title):
                        jelClassification_list = [keyword.text for keyword in keywords_div]
                        article['jelClassification'] = ','.join(jelClassification_list[1:])
                articles.append(article)
                #time.sleep(np.random.randint(0,5)) #在2秒~10秒钟随意切换数值，降低爬取频率，防止被封杀
    
    articles_df = pd.DataFrame(articles)
    articles_df.to_csv('{}{}-{}.csv'.format(url_sub,volume1,volume2+1), encoding='utf-8')   
    

In [ ]:
def saveInfoToFile3(volume1,volume2,url_sub,url_origi):
    '''
    解析数据，返回需要的数据
    volume1:起始期目号, 最大的
    volume2:结束期目号+1
    url_sub:网址中间类别
    url_c：TURE:url组后是C，否则是S1
    '''
    articles = []
    Keywords_title = "Keywords"
    jelClassification_title = "JEL"

    for volume_tmp in range(volume1,volume2,-1):
        for page_tmp in range(page,0,-1):
            url = url_origi.format(url_sub,str(volume_tmp))
            print(url)
            soup = getinfo(url)    
            lis = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')
            for li in lis:
                article={}
                article['title'] = li.find('span',class_='js-article-title').text
                article['link'] = 'https://www.sciencedirect.com'+li.find('a', class_='anchor article-content-title u-margin-xs-top u-margin-s-bottom')['href']
                try:
                    article['authors'] = li.find('div',class_='text-s u-clr-grey8 js-article__item__authors').text
                except AttributeError:
                    article['authors'] = ''
                soup1 = getinfo(article['link'])
                try:
                    article['abstract']=soup1.findAll('div', class_="abstract author")[0].find_all('div')[0].text
                except:
                    article['abstract']=''
                article['期刊页码']= soup1.findAll('div',class_='text-xs')[0].text
                article['jelClassification'] = '无'
                keywords_divs=soup1.findAll('div', class_="keywords-section")
                for keywords_div in  keywords_divs:           
                    #对当前节点前面的标签和字符串进行查找
                    div_title = keywords_div.find_all('h2')[0].text
                    if(Keywords_title in div_title):
                        keywords_list = [keyword.text for keyword in keywords_div]
                        article['keywords'] = ','.join(keywords_list[1:])
                    if(jelClassification_title in div_title):
                        jelClassification_list = [keyword.text for keyword in keywords_div]
                        article['jelClassification'] = ','.join(jelClassification_list[1:])
                articles.append(article)
                #time.sleep(np.random.randint(0,5)) #在2秒~10秒钟随意切换数值，降低爬取频率，防止被封杀
    
    articles_df = pd.DataFrame(articles)
    articles_df.to_csv('{}{}-{}.csv'.format(url_sub,volume1,volume2+1), encoding='utf-8')   

In [ ]:
def saveInfoToFile4(volume1,volume2,url_origi):
    '''
    解析数据，返回需要的数据
    volume1:起始期目号, 最大的
    volume2:结束期目号+1
    '''
    articles = []
    Keywords_title = "Keywords"
    jelClassification_title = "JEL"

    for volume_tmp in range(volume1,volume2,-1):
        for page_tmp in range(page,0,-1):
            url = url_origi
            print(url)
            soup = getinfo(url)    
            lis = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')
            for li in lis:
                article={}
                article['title'] = li.find('span',class_='js-article-title').text
                article['link'] = 'https://www.sciencedirect.com'+li.find('a', class_='anchor article-content-title u-margin-xs-top u-margin-s-bottom')['href']
                try:
                    article['authors'] = li.find('div',class_='text-s u-clr-grey8 js-article__item__authors').text
                except AttributeError:
                    article['authors'] = ''
                soup1 = getinfo(article['link'])
                try:
                    article['abstract']=soup1.findAll('div', class_="abstract author")[0].find_all('div')[0].text
                except:
                    article['abstract']=''
                article['期刊页码']= soup1.findAll('div',class_='text-xs')[0].text
                article['jelClassification'] = '无'
                keywords_divs=soup1.findAll('div', class_="keywords-section")
                for keywords_div in  keywords_divs:           
                    #对当前节点前面的标签和字符串进行查找
                    div_title = keywords_div.find_all('h2')[0].text
                    if(Keywords_title in div_title):
                        keywords_list = [keyword.text for keyword in keywords_div]
                        article['keywords'] = ','.join(keywords_list[1:])
                    if(jelClassification_title in div_title):
                        jelClassification_list = [keyword.text for keyword in keywords_div]
                        article['jelClassification'] = ','.join(jelClassification_list[1:])
                articles.append(article)
                #time.sleep(np.random.randint(0,5)) #在2秒~10秒钟随意切换数值，降低爬取频率，防止被封杀
    
    articles_df = pd.DataFrame(articles)
    articles_df.to_csv('{}{}-{}.csv'.format(url_sub,volume1,volume2+1), encoding='utf-8') 

In [ ]:
parser = argparse.ArgumentParser(__doc__)
parser.add_argument("--url_issue", type=ast.literal_eval, default=True, help="url have issue")
parser.add_argument("--url_c", type=ast.literal_eval, default=True, help="end of url is C")
parser.add_argument("--url_s1", type=ast.literal_eval, default=True, help="end of url is S1")
parser.add_argument("--url_sub", type=str, default=None, help="url intermediate category.")
parser.add_argument("--page", type=int, default=None, help="maximal page.")
parser.add_argument("--volume1", type=int, default=None, help="beginning volume.")i
parser.add_argument("--volume2", type=int, default=None, help=" end volume.")
args = parser.parse_args()


In [ ]:
if __name__ == '__main__':
   volume1,volume2 = args.volume1,args.volume2
   a =saveInfoToFile1(volume1,volume2)
   a_df = pd.DataFrame(a)
   a_df.to_csv('文献信息{}-{}.csv'.format(volume1,volume2+1), encoding='utf-8')



In [ ]:
if __name__ == '__main__':
   volume1,volume2 = 22,14
   a =saveInfoToFile1(volume1,volume2)
   a_df = pd.DataFrame(a)
   a_df.to_csv('文献信息{}-{}.csv'.format(volume1,volume2+1), encoding='utf-8')

In [9]:
if __name__ == '__main__':
    volume1,volume2 = 28,27
    page = 12
    url_sub = 'world-development'
    url_origi = 'https://www.sciencedirect.com/journal/{}/vol/{}/issue/{}'
    saveInfoToFile2(volume1,volume2,url_sub,page,url_origi)
    

https://www.sciencedirect.com/journal/world-development/vol/28/issue/12


ConnectionError: HTTPSConnectionPool(host='www.sciencedirect.com', port=443): Max retries exceeded with url: /science/article/pii/S0305750X00000784 (Caused by ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')))

In [ ]:
nohup python /home/aistudio/sciencedirect.py --url_c True --url_sub 'world-development' --volume1 140 --volume2 100 &
nohup python /home/aistudio/sciencedirect.py --url_c True --url_sub 'world-development' --volume1 100 --volume2 70 &
nohup python /home/aistudio/sciencedirect.py --url_c True --url_sub 'world-development' --volume1 70 --volume2 40 &
nohup python /home/aistudio/sciencedirect.py --url_s1 True --url_sub 'world-development' --volume1 64 --volume2 63 &

In [ ]:
if __name__ == '__main__':
    
    volume1,volume2 = args.volume1,args.volume2
    page = args.page
    url_sub = args.url_sub
    saveInfoToFile2(volume1,volume2,url_sub,page)

In [ ]:
if __name__ == '__main__':
    if args.url_issue:
        url_origi = 'https://www.sciencedirect.com/journal/{}/vol/{}/issue/{}'
        saveInfoToFile2(args.volume1,args.volume2,args.url_sub,args.page,url_origi)
    elif args.url_c:
        url_origi = 'https://www.sciencedirect.com/journal/{}/vol/{}/suppl/C'
        saveInfoToFile3(args.volume1,args.volume2,args.url_sub,url_origi)
    elif args.url_s1:
        url_origi = 'https://www.sciencedirect.com/journal/{}/vol/{}/suppl/S1'
        saveInfoToFile3(args.volume1,args.volume2,args.url_sub,url_origi)
    else:
        url_origi = args.url_origi
        saveInfoToFile4(args.volume1,args.volume2,url_origi)
    

In [ ]:
!zip -r articles.zip /home/aistudio/*.csv

# 合并csv文件

In [4]:

#待搜索的目录路径
path = "/home/aistudio/result_mid"
#待搜索的名称
filename = ".csv"
#定义保存结果的数组
result = []
def findfiles(path,filename):
    #在这里写下您的查找文件代码吧！
    for maindir, subdir, file_name_list in os.walk(path):
        for filename1 in file_name_list:
            #print(filename1)
            if filename in filename1:
                apath = os.path.join(maindir, filename1)#合并成一个完整路径
                #print(apath)
                result.append(apath)
    return result


In [12]:
def add_url_sub(filename,url_sub):  
    urls = findfiles('/home/aistudio/result_mid1', filename)
    dfs = [pd.read_csv(url,header=0,index_col=0,encoding='utf-8') for url in urls]
    for df in dfs:
        df['url_sub'] = url_sub
    outdf =pd.concat(dfs, axis = 0, ignore_index=True)
    return outdf



In [13]:

outdf =pd.concat([add_url_sub(filename,url_sub) for  (filename,url_sub) in zip(["world-development",'economia','文献信息'],["world-development",'economia','Journal of Public Economics'])], axis = 0, ignore_index=True)

In [19]:
outdf=outdf.drop_duplicates()

In [20]:
#获取当天的日期,并进行格式化,用于后面文件命名，格式:20200420
today = datetime.date.today().strftime('%Y%m%d')  
outdf.to_csv('/home/aistudio/result/文献信息{}.csv'.format(today), encoding='utf-8')

In [23]:
outdf['url_sub'].nunique()

3

In [44]:
print(len(urls))

39


In [45]:
dfs = [pd.read_csv(url,header=0,index_col=0,encoding='utf-8') for url in urls]
outdf =pd.concat(dfs, axis = 0, ignore_index=True)
outdf.to_csv('/home/aistudio/result/文献信息.csv', encoding='utf-8')

In [47]:
!wc -l result/文献信息.csv

10565 result/文献信息.csv


In [21]:
!wc -l result/文献信息20210714.csv

26560 result/文献信息20210714.csv


In [22]:
!head -n 5 result/文献信息20210714.csv

,title,link,authors,abstract,期刊页码,jelClassification,keywords,url_sub
0,Editorial: Human capital and human capability,https://www.sciencedirect.com/science/article/pii/S0305750X97100146,Amartya Sen,,"Volume 25, Issue 12, December 1997, Pages 1959-1961",无,,world-development
1,“Unanticipated consequences” in anti-poverty programs,https://www.sciencedirect.com/science/article/pii/S0305750X9700106X,Robert Klitgaard,"Many anti-poverty programs have unanticipated consequences. A typology is provided, and one of its categories is illustrated with examples: insufficient attention to the economics of the institutions that implement anti-poverty programs. This in turn leads to practical suggestions for improving these institutions, especially through better information and incentives.","Volume 25, Issue 12, December 1997, Pages 1963-1972",无,"Poverty,public management,incentives,adjustment",world-development
2,Testing the limits of privatization: Argentine railroads,https://www.sciencedirect.com/s

In [4]:
df = pd.read_csv('/home/aistudio/result/文献信息.csv',encoding='utf-8')
df[0:5]

,Unnamed: 0,title,link,authors,abstract,期刊页码,jelClassification,keywords
0,0,Post-Conflict Reconstruction and the Challenge...,https://www.sciencedirect.com/science/article/...,Graciana del Castillo,Countries in post-conflict transitions have to...,"Volume 29, Issue 12, December 2001, Pages 1967...",无,"Latin America,El Salvador,political economy,po..."
1,1,One Kind of Freedom: Poverty Dynamics in Post-...,https://www.sciencedirect.com/science/article/...,"Michael R. Carter, Julian May",A 1993 South African living standards survey d...,"Volume 29, Issue 12, December 2001, Pages 1987...",无,"income dynamics,poverty measures,South Africa"
2,2,Is Devolution Democratization?,https://www.sciencedirect.com/science/article/...,Nandini Sundar,Recent attempts at introducing new forms of go...,"Volume 29, Issue 12, December 2001, Pages 2007...",无,"devolution,social capital,state,forestry,Asia,..."
3,3,The Manufacture of Popular Perceptions of Scar...,https://www.sciencedirect.com/science/article/...,Lyla Mehta,This paper critically examines some narratives...,"Volume 29, Issue 12, December 2001, Pages 2025...",无,"Asia,India,water scarcity,dams,narratives of s..."
4,4,Institutional Pluralism and Housing Delivery: ...,https://www.sciencedirect.com/science/article/...,"Bishwapriya Sanyal, Vinit Mukhija",This paper demonstrates how institutional plur...,"Volume 29, Issue 12, December 2001, Pages 2043...",无,"Asia,India,housing,conflict,NGOs,slum redevelo..."


In [24]:
df['url_sub'] = df['link'].apply(lambda x: x.split('/')[-2])

In [25]:
df.loc[1:5,['link','url_sub']]

,link,url_sub
1,https://www.sciencedirect.com/science/article/...,science
2,https://www.sciencedirect.com/science/article/...,science
3,https://www.sciencedirect.com/science/article/...,science
4,https://www.sciencedirect.com/science/article/...,science
5,https://www.sciencedirect.com/science/article/...,science


# https://link.springer.com/journal/10018/volumes-and-issues

In [1]:
import sys 
sys.path.append('/home/aistudio/external-libraries')
import json
import re
import requests
import datetime
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import random
import time #时间处理模块
import argparse
import copy
import ast
import os

In [2]:
def get_headers():
    user_agent = [
        "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
        "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
        "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
        "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
        "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
        "Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
        "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
        "Mozilla/5.0 (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
        "Mozilla/5.0 (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
        "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
        "Mozilla/5.0 (BlackBerry; U; BlackBerry 9800; en) AppleWebKit/534.1+ (KHTML, like Gecko) Version/6.0.0.337 Mobile Safari/534.1+",
        "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.0; U; en-US) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/233.70 Safari/534.6 TouchPad/1.0",
        "Mozilla/5.0 (SymbianOS/9.4; Series60/5.0 NokiaN97-1/20.0.019; Profile/MIDP-2.1 Configuration/CLDC-1.1) AppleWebKit/525 (KHTML, like Gecko) BrowserNG/7.1.18124",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0; HTC; Titan)",
        "UCWEB7.0.2.37/28/999",
        "NOKIA5700/ UCWEB7.0.2.37/28/999",
        "Openwave/ UCWEB7.0.2.37/28/999",
        "Mozilla/4.0 (compatible; MSIE 6.0; ) Opera/UCWEB7.0.2.37/28/999",
        "Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25"
    ]

    headers = {
        
        
        
        'User-Agent': random.choice(user_agent),
        
    }

    return headers

In [3]:
def getinfo(url):
    '''
    请求url接口，返回soup信息
    '''
    try:
        re = requests.get(url, headers=get_headers())
        
        soup = BeautifulSoup(re.text, 'html.parser')
        return soup
    except Exception as e:
        print(e) 

In [6]:
def saveInfoToFile(volume1,volume2,url_sub,page1,page2):
    articles = []
    Keywords_title = "Keywords"
    jelClassification_title = "JEL Classification"
    for volume_tmp in range(volume1,volume2,-1):
        for page_tmp in range(page1,page2,-1):    
            url = url_origi.format(url_sub,str(volume_tmp),str(page_tmp))
            print(url)
            time.sleep(np.random.randint(1,3))
            soup = getinfo(url)

            page_journey =  soup.find('h1', class_="u-mb-8").text

            lis = soup.find_all('li', class_='c-list-group__item')
            for li in lis:
                article={}
                article['title'] = li.find('h3',class_='c-card__title').text.strip()
                article['link'] = li.find('a')['href']
                creator = li.find_all('li',itemprop='creator')
                article['authors'] = []
                for author in creator:
                    article['authors'].append(author.find('span',itemprop='name').text)
                article['authors']=','.join(article['authors'])
                article['page'] =  page_journey + ','+ li.find_all('li',class_="c-meta__item")[-1].text
                time.sleep(np.random.randint(1,3))
                soup1 = getinfo(article['link'])
                try:
                    article['abstract']=soup1.findAll('div', class_="c-article-section__content")[0].text
                except:
                    article['abstract']=''

                keywords_list=[]
                jels_list=[]
                try:
                    subjects = soup1.find_all('li', class_="c-article-subject-list__subject")
                except:
                    article['keywords']=''
                    article['jelClassification']=''
                    articles.append(article)
                    continue
                for subject in subjects:
                    #对当前节点前面的标签和字符串进行查找
                    subject_title= subject.find_previous('h3', class_="c-article__sub-heading")
                    if(Keywords_title in subject_title):
                        keywords_list.append(subject.text)
                    if(jelClassification_title in subject_title):
                        jels_list.append(subject.text)

                article['keywords'] = ','.join(keywords_list)
                article['jelClassification'] = ','.join(jels_list)    

                articles.append(article)
    
    articles_df = pd.DataFrame(articles)
    articles_df['url_sub'] = url_sub
    articles_df.to_csv('/home/aistudio/result_mid/{}-{}-{}.csv'.format(url_sub,volume1,volume2+1), encoding='utf-8')

In [7]:
if __name__ == '__main__':
    url_origi = 'https://link.springer.com/journal/{}/volumes-and-issues/{}-{}'
    #saveInfoToFile(volume1,volume2,url_sub,page1,page2)
    saveInfoToFile(23,22,'10018',3,1)

https://link.springer.com/journal/10018/volumes-and-issues/23-3
https://link.springer.com/journal/10018/volumes-and-issues/23-2


https://academic.oup.com/wber/article-abstract/35/2/287/5607661?redirectedFrom=fulltext

In [4]:
url = 'https://academic.oup.com/wber/article-abstract/35/2/287/5607661?redirectedFrom=fulltext'
soup = getinfo(url)


In [12]:
 ','.join([keyword.text for keyword in soup.findAll('a', class_="kwd-part kwd-main")])

'sanitation,urbanization,religion,culture'

In [24]:
soup.findAll('div', class_="article-metadata")[0].text.strip()[4:].replace('\n',',')

'I12 - Health Behavior,O15 - Human Resources; Human Development; Income Distribution; Migration,Z12 - Religion'